# Interactive Visualization of Inferences (InVizIn)


In [ ]:
import dash
import dash_table
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html

In [ ]:
def createTable(df):
    table_header_style = {
        "backgroundColor": "rgb(2,21,70)",
        "color": "white",
        "textAlign": "center",
    }

    tab = dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("rows"),
        style_header=table_header_style,
        style_table={'maxHeight': '40%',
                    'maxWidth': '50%'},
        fixed_rows={ 'headers': True, 'data': 0 },
    )
    return tab

In [ ]:
def createScatter(df):
    dataTemplate = dict(
        x=df[df.columns[0]],
        y=df[df.columns[1]],  #text=df.columns[0],
        mode='markers',
        opacity=0.7,
        marker={
            'size': 15,
            'line': {'width': 0.5, 'color': 'white'}
        },
        name=df.columns[0])
    
    dataDict = []
    dataDict.append(dataTemplate)
        
    scatterGraph = dcc.Graph(
        id='scatter',
        figure={
            'data': dataDict,
            'layout': dict(
                xaxis={'title': df.columns[0]},
                yaxis={'title': df.columns[1]},
                hovermode='closest')
        }
    )
    return scatterGraph

In [ ]:
df = pd.read_csv('C:\\Users\\212613144\\Repository\\DARPA-ASKE-TA1-Ext\\Datasets\\Force_dataset.csv', header = [0,1])

colNames = []
for i in range(len(df.columns)):
     colNames.append(df.columns[i][0])
df.columns = colNames

app = dash.Dash(__name__)
table = createTable(df)
graph = createScatter(df)
app.layout = html.Div(
                className="container",
                children=[
                    html.Div([
                            html.Label(
                                [
                                    html.Div(["Mass"]),
                                    dcc.Input(
                                        id="mass-input",
                                        placeholder="Enter a value...",
                                        type="number",
                                        value=1.0,
                                        min=0.0,
                                        max=10.0,
                                    ),
                                ]
                            ),
                            html.Label(
                                [
                                    html.Div(["Acceleration"]),
                                    dcc.Input(
                                        id="acc-input",
                                        placeholder="Enter a value...",
                                        type="number",
                                        value=0.5,
                                        min=-10.0,
                                        max=10.0,
                                    ),
                                ]
                            ),
                    ]
                    ),
                    html.Div(
                        className="row",
                        style={},
                        children=[
                            html.Div(className="table",
                                     children=[table]
                            ),
                            
                            html.Div(className="graph",
                                     children=[graph]
                            )
                        ]
                    )
                ]
)
                            
                            
app.run_server(debug=False)
    

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
    row=1, col=2
)

fig.update_layout(height=600, width=800, title_text="Subplots")
fig.show()

In [ ]:
app = dash.Dash(__name__)
graph = createScatter(df)
app.layout = html.Div(children=[graph])

if __name__ == '__main__':
    app.run_server(debug=False)

In [1]:
#imports needed for demonstration

#for communicating with services
import requests

#URL to interact with build
url_build = 'http://localhost:12345/darpa/aske/kchain/build'

#URL to interact with append service
url_append = 'http://localhost:12345/darpa/aske/kchain/append'

#URL to interact with evaluate service
url_evaluate = 'http://localhost:12345/darpa/aske/kchain/evaluate'

In [3]:
inputPacket = {
"inputVariables": [
    {
        "name": "mach",
        "type": "float"
    },
    {
        "name": "gamma",
        "type": "float",
        "value":"1.4"
    }
],
"outputVariables": [
    {
      "name": "aflow",
      "type": "float"
    },
    {
      "name": "fac1",
      "type": "float"
    },
    {
      "name": "fac2",
      "type": "float"
    }
],
"equationModel" : """#   Utility to get the corrected airflow per area given the Mach number 
    fac2 = (gamma + 1.0) / (2.0 * (gamma - 1.0))
    fac1 = tf.math.pow((1.0 + 0.5 * (gamma - 1.0) * mach * mach), fac2)
    number = 0.50161 * tf.math.sqrt(gamma) * mach / fac1
    aflow = number
""",
"modelName" : "getAir"
}
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/getAir',
 'modelType': 'Physics',
 'trainedState': 0}

In [4]:
evalPacket = {
"inputVariables": [
    {
        "name": "mach",
        "type": "float",
        "value" : "0.9",
        #"minValue": "0.0",
        #"maxValue": "3.0"
    },
    {
        "name": "gamma",
        "type": "float",
        "value" : "1.4",
        #"minValue": "1.01",
        #"maxValue": "2.0"
    },
    
],
"outputVariables": [
    {
      "name": "aflow",
      "type": "float"
    },
    {
      "name": "fac1",
      "type": "float"
    },
    {
      "name": "fac2",
      "type": "float"
    }
],
"modelName" : "getAir"
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'error': '',
 'inputVariables': [{'name': 'mach', 'type': 'float', 'value': '0.9'},
  {'name': 'gamma', 'type': 'float', 'value': '1.4'}],
 'outputVariables': [{'name': 'aflow',
   'type': 'float',
   'value': '[0.3404508]'},
  {'name': 'fac1', 'type': 'float', 'value': '[1.5689833]'},
  {'name': 'fac2', 'type': 'float', 'value': '[3.0000002]'}]}

In [4]:
r.status_code

200

In [5]:
from plotly.subplots import make_subplots
import plotly.colors as colors 
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import copy

In [6]:
def getOutputValue(evalPacket, url_evaluate, inputVal, index):
    if inputVal is not None:
        evalPacket['inputVariables'][index]['value'] = str(inputVal)
    r = requests.post(url_evaluate, json=evalPacket)
    rj = r.json()
    assert r.status_code == 200
    #print(rj)
    outVals = []
    for ii in range(len(rj['outputVariables'])):
        outVals.append(float(rj['outputVariables'][ii]['value'][1:-1]))

    return outVals

In [7]:
def getOutputDataframe(evalPacket, url_evaluate, X, index):
    df = pd.DataFrame()
    for inputVal in X:
        outVals = getOutputValue(evalPacket, url_evaluate, inputVal, index)
        dat = {}
        dat[evalPacket['inputVariables'][index]['name']] = inputVal #X
        for ix, outVal in enumerate(outVals):
            dat[evalPacket['outputVariables'][ix]['name']] = outVal
        df = df.append(dat, ignore_index=True)
        
    return df

In [8]:
def getMinMax(pck, index):
    frac = 0.1
    if 'minValue' in pck['inputVariables'][index].keys():
        MIN = float(pck['inputVariables'][index]['minValue'])
    else:
        MIN = float(pck['inputVariables'][index]['value'])*(1 - frac)
        
    if 'maxValue' in pck['inputVariables'][index].keys():
        MAX = float(pck['inputVariables'][index]['maxValue'])
    else:
        MAX = float(pck['inputVariables'][index]['value'])*(1 + frac)
    
    return MIN, MAX

In [9]:
def createSensitivityGraphOAT(evalPacket):
    pck = copy.deepcopy(evalPacket)
    NUM = 20

    refDat = {}
    for ii, inputVariable in enumerate(pck['inputVariables']):
        inVal = float(inputVariable['value'])
        refDat[inputVariable['name']]=inVal

    outVals = getOutputValue(evalPacket, url_evaluate, inputVal=None, index=None)
    for ix, outVal in enumerate(outVals):
        refDat[pck['outputVariables'][ix]['name']] = outVal

    fig = make_subplots(rows=len(pck['outputVariables']), cols=len(pck['inputVariables']), 
                        shared_xaxes=True, horizontal_spacing=0.075, vertical_spacing=0.05) #shared_yaxes= True, shared_xaxes=True, 
    for ii, inputVariable in enumerate(pck['inputVariables']):
        #only one input is changed while others stay at base
        MIN, MAX = getMinMax(pck, ii)
        X = np.linspace(MIN, MAX, num=NUM)
        df = getOutputDataframe(copy.deepcopy(pck), url_evaluate, X, index = ii)
        for jj, outputVariable in enumerate(pck['outputVariables']):
            fig.add_trace(go.Scatter(x = X, y=df[outputVariable['name']], 
                                     mode="lines",name = inputVariable['name'],
                                     marker = {"size": 10}),
                          row=jj+1, col=ii+1)
            fig.add_trace(go.Scatter(x = [refDat[inputVariable['name']]], 
                                     y = [refDat[outputVariable['name']]],
                                     name = "Reference", mode="markers", 
                                     marker = {"symbol":"diamond-open","size": 10, "color": "black"}),
                          row=jj+1, col=ii+1)
            if ii == 0:
                fig.update_yaxes(title_text=outputVariable['name'], row=jj+1, col=ii+1, hoverformat=".2f")
            else:
                fig.update_yaxes(row=jj+1, col=ii+1, hoverformat=".2f")
            if jj == len(pck['outputVariables'])-1:
                fig.update_xaxes(title_text=inputVariable['name'], row=jj+1, col=ii+1, hoverformat=".2f")
            else:
                fig.update_xaxes(row=jj+1, col=ii+1, hoverformat=".2f")

    fig.update_layout(title='Sensitivity: One-at-a-time Parametric Analysis', showlegend=False)
    fig.show()
    return fig

In [13]:
import dash
import dash_core_components as dcc

app = dash.Dash(
    __name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}]
)

fig = createSensitivityGraphOAT(evalPacket)
app.layout = dcc.Graph(figure=fig)
#app.run_server(debug=False, port=7779)

In [11]:
evalPacket = {
                  "inputVariables": [
                    {
                        "name": "u0d",
                        "type": "float",
                        "value": "100.0"
                    },
                    {
                        "name": "altd",
                        "type": "float",
                        "value": "10000.0"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "fsmach",
                      "type": "float"
                    },
                    {
                      "name": "a0",
                      "type": "float"
                    },
                    {
                      "name": "cpair",
                      "type": "float"
                    } 
                  ],
                    "modelName" : "getResponse"
                 }
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'error': '',
 'inputVariables': [{'name': 'u0d', 'type': 'float', 'value': '100.0'},
  {'name': 'altd', 'type': 'float', 'value': '10000.0'}],
 'outputVariables': [{'name': 'fsmach',
   'type': 'float',
   'value': '[0.13607623]'},
  {'name': 'a0', 'type': 'float', 'value': '[1077.8273]'},
  {'name': 'cpair', 'type': 'float', 'value': '[0.23808765]'}]}

# Local Sensitivity Analysis with Gradients

In [83]:
inputPacket = {
                  "inputVariables": [
                    {
                      "name": "u0d",
                      "type": "float"
                    },
                    {
                      "name": "altd",
                      "type": "float"
                    },
                    {
                        "name": "fsmach",
                        "type": "float"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "grad_fsmach_u0d",
                      "type": "float"
                    },
                    {
                      "name": "grad_fsmach_altd",
                      "type": "float"
                    },
                    {
                      "name": "sens_fsmach_u0d",
                      "type": "float"
                    },
                    {
                      "name": "sens_fsmach_altd",
                      "type": "float"
                    }
                  ],
                   "equationModel" : """
    grad_fsmach_u0d = tf.gradients(fsmach, u0d, stop_gradients = [u0d])[0]
    grad_fsmach_altd = tf.gradients(fsmach, altd, stop_gradients = [altd])[0]
    sens_fsmach_u0d = tf.math.abs(grad_fsmach_u0d)*u0d/fsmach
    sens_fsmach_altd = tf.math.abs(grad_fsmach_altd)*altd/fsmach
                   """,
                   "modelName" : "grad_fsmach",
                   "targetModelName" : "getResponse"
                 }
#send request to build model
r = requests.post(url_append, json=inputPacket)

#see the response
r.json()

{'metagraphLocation': '../models/getResponse', 'modelType': 'Physics'}

In [84]:
evalPacket = {
                  "inputVariables": [
                    {
                        "name": "u0d",
                        "type": "float",
                        "value": "100.0"
                    },
                    {
                        "name": "altd",
                        "type": "float",
                        "value": "10000.0"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "fsmach",
                      "type": "float"
                    },
                    {
                      "name": "grad_fsmach_u0d",
                      "type": "float"
                    },
                    {
                      "name": "grad_fsmach_altd",
                      "type": "float"
                    },
                    {
                      "name": "sens_fsmach_u0d",
                      "type": "float"
                    },
                    {
                      "name": "sens_fsmach_altd",
                      "type": "float"
                    } 
                  ],
                    "modelName" : "getResponse"
                 }
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'error': '',
 'inputVariables': [{'name': 'u0d', 'type': 'float', 'value': '100.0'},
  {'name': 'altd', 'type': 'float', 'value': '10000.0'}],
 'outputVariables': [{'name': 'fsmach',
   'type': 'float',
   'value': '[0.13607623]'},
  {'name': 'grad_fsmach_u0d', 'type': 'float', 'value': '[0.00136076]'},
  {'name': 'grad_fsmach_altd', 'type': 'float', 'value': '[5.0148174e-07]'},
  {'name': 'sens_fsmach_u0d', 'type': 'float', 'value': '[1.]'},
  {'name': 'sens_fsmach_altd', 'type': 'float', 'value': '[0.036853]'}]}